In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import keras_tuner as kt
import os

from scipy.io import loadmat
from keras_tuner import HyperParameter as hp

In [ ]:
# Define fonts and fontsize for plotting
plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
fontsize = 15

In [ ]:
# Define necessary functions
# Function to get displacement and void data
def get_data(file_directory, void_number):

    # Load the displacement data
    training_displacement_data = loadmat(os.path.join(file_directory, f'void_{void_number}_training_displacement_data_normalized.mat'))['displacement_data']
    validation_displacement_data = loadmat(os.path.join(file_directory, f'void_{void_number}_validation_displacement_data_normalized.mat'))['displacement_data']

    # Load the void data
    training_void_data = loadmat(os.path.join(file_directory, f'void_{void_number}_training_void_data.mat'))['void_data']
    validation_void_data = loadmat(os.path.join(file_directory, f'void_{void_number}_validation_void_data.mat'))['void_data']

    return training_displacement_data, validation_displacement_data, training_void_data, validation_void_data

In [ ]:
# Function to return searched optimizers
def return_optimizer(optimizer_search, learning_rate_search):

    # If else block to return the optimizer and learning rate
    if optimizer_search == 'adam':

        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'nadam':

        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'rmsprop':

        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'adadelta':

        optimizer = tf.keras.optimizers.Adadelta(learning_rate=learning_rate_search)
    
    elif optimizer_search == 'adagrad':

        optimizer = tf.keras.optimizers.Adagrad(learning_rate=learning_rate_search)

    elif optimizer_search == 'adamax':

        optimizer = tf.keras.optimizers.Adamax(learning_rate=learning_rate_search)
    
    return optimizer

In [ ]:
# Function to build model for hyperparameter search
def build_model(hp):

    # Model Architecture Stage
    # Activation Functions
    activation_function_search = hp.Choice('activation_function', values=['relu', 'LeakyReLU'])

    # Number of layers
    number_of_layers_search = hp.Int('number_of_layers', min_value=1, max_value=3, step=1)

    # Kernel Initializer
    kernel_initializer_search = hp.Choice('kernel_initializer', values=['glorot_normal', 'he_normal', 'he_uniform', 'glorot_uniform'])
    
    # Kernel Sizes
    kernel_size_search = hp.Int('kernel_size', min_value=10, max_value=100, step=10)

    # Learning Rate
    learning_rate_search = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])

    # Optimizer
    optimizer_search = hp.Choice('optimizer', values=['adam', 'nadam', 'rmsprop'])

    # Define a sequential model
    model = tf.keras.models.Sequential()

    # Add the input layer
    model.add(tf.keras.layers.Input(shape=input_shape))

    # Add the convolutional hidden layers
    for i in range(number_of_layers_search):
        
        model.add(tf.keras.layers.Conv1D(filters=hp.Int(f'conv_{i+1}_filters', min_value=10, max_value=200, step=10),
                                         kernel_size=kernel_size_search, 
                                         activation=activation_function_search, 
                                         kernel_initializer=kernel_initializer_search,
                                         padding='same'))

        # Add Batch Normalization layer
        model.add(tf.keras.layers.BatchNormalization())

    # Add a flatten layer
    model.add(tf.keras.layers.Flatten())
        
    # Add the output layer
    model.add(tf.keras.layers.Dense(units=output_shape, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=return_optimizer(optimizer_search, learning_rate_search),
                  loss = tf.keras.losses.BinaryCrossentropy(name='binary_crossentropy'), 
                  metrics = tf.keras.metrics.Precision(name='precision'))
    
    return model

In [ ]:
# Define directories
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
grandparent_directory = os.path.dirname(parent_directory)

# Normalized data directory
normalized_data_directory = os.path.join(grandparent_directory, 'data', 'normalized')

# Hyperparameter search directory
hyperparameter_search_directory = os.path.join(grandparent_directory, 'cnn')
hyperparameter_search_folder = 'hyperparameter_search'

# # If only this folder exists, then delete it
# if os.path.exists(hyperparameter_search_directory):

#     os.chdir(hyperparameter_search_directory)

#     # If the folder exists, delete it
#     if os.path.exists(hyperparameter_search_folder):

#         os.system(f'rm -r {hyperparameter_search_folder}')
        
# Training results directory
training_results_directory = os.path.join(grandparent_directory, 'cnn', 'training_results')

In [ ]:
# Get the normalized data for all three voids
void_0_training_displacement_data, void_0_validation_displacement_data, void_0_training_void_data, void_0_validation_void_data = get_data(normalized_data_directory, 0)
void_1_training_displacement_data, void_1_validation_displacement_data, void_1_training_void_data, void_1_validation_void_data = get_data(normalized_data_directory, 1)
void_2_training_displacement_data, void_2_validation_displacement_data, void_2_training_void_data, void_2_validation_void_data = get_data(normalized_data_directory, 2)

In [ ]:
# Vertically stack the training, validation and test data
# Displacement
training_displacement_data = np.vstack((void_0_training_displacement_data, void_1_training_displacement_data, void_2_training_displacement_data))
validation_displacement_data = np.vstack((void_0_validation_displacement_data, void_1_validation_displacement_data, void_2_validation_displacement_data))

# Void
training_void_data = np.vstack((void_0_training_void_data, void_1_training_void_data, void_2_training_void_data))
validation_void_data = np.vstack((void_0_validation_void_data, void_1_validation_void_data, void_2_validation_void_data))

In [ ]:
# Randomize the dataset using the same seed
np.random.seed(42)

# Randomize the training data
random_indices = np.random.permutation(training_displacement_data.shape[0])
training_displacement_data = training_displacement_data[random_indices]
training_void_data = training_void_data[random_indices]

# Randomize the validation data
random_indices = np.random.permutation(validation_displacement_data.shape[0])
validation_displacement_data = validation_displacement_data[random_indices]
validation_void_data = validation_void_data[random_indices]

In [ ]:
# Define variables that remain constant during the training
input_shape = training_displacement_data.shape[1:]
output_shape = training_void_data.shape[1]

In [ ]:
# Use the hyperband tuner to search for the best hyperparameters
hyperband_tuner = kt.Hyperband(build_model,
                     objective=kt.Objective('val_loss', direction = 'min'),
                     max_epochs=1000,
                     directory=hyperparameter_search_directory,
                     project_name=hyperparameter_search_folder)

In [ ]:
# Define callbacks
# Early stopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [ ]:
# Display search space summary
hyperband_tuner.search_space_summary()

In [ ]:
# Search for the best hyperparameters
hyperband_tuner.search(x=training_displacement_data,
                       y=training_void_data,
                       epochs=100,
                       validation_data=(validation_displacement_data, validation_void_data),
                       callbacks=[early_stopping])

In [ ]:
# Get the best trained model
best_model = hyperband_tuner.get_best_models(num_models=1)[0]

# Save the best model
best_model.save(os.path.join(training_results_directory, 'best_model.h5'))

# Print message to the user
print('Hyperparameter search completed successfully.')